In [1]:
import pandas as pd

df = pd.read_csv('/content/sample_data/final_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Depression', axis=1)
y_train = train_set['Depression'].copy()

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [3]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=1000, max_depth=14, n_jobs=-1, random_state=42)),
        ('nonlinear_svc', SVC(kernel='poly', degree=3, C=1, random_state=42, probability=True)),
    ],
    voting='soft'
)

In [4]:
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_train)

In [5]:
# cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(voting_clf, X_train, y_train, cv=3, scoring='accuracy')

array([0.93614072, 0.93979211, 0.93683369])

In [6]:
# split target and features
X_test = test_set_from_train.drop('Depression', axis=1)
y_test = test_set_from_train['Depression'].copy()
X_test = scaler.transform(X_test)

In [7]:
y_pred = voting_clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

array([0.93720682, 0.93646055, 0.93720682])

In [10]:
# save final prediction
test_set = pd.read_csv('/content/sample_data/final_test.csv')
test_set = scaler.transform(test_set)
y_pred = voting_clf.predict(test_set)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Depression': y_pred})
final_prediction.to_csv('voting_clf_final_prediction.csv', index=False)